In [92]:
txt = '''This is an example sentence, ok?

And, see! This is another (example sentence), isn't more.'''

In [93]:
## Make nested word_list
import re
from graphbrain.parsers import *

parser = create_parser(lang='en')

def split_words(text=''):
    words = re.split('(\W)',text)
    words = list(filter(lambda w: (w != ''), words))
    last = ''
    result = []
    for word in words:
        if last == "'":
            match = re.match('s|re|m|ve|d|t|ll|clock', word, re.I)
            if match:
                word = result.pop() + word
                if match[0] == 't' and re.match('n',result[-1][-1], re.I):
                    last = list(result.pop())
                    word = last.pop() + word
                    result.append(''.join(last)) 


        result.append(word)
        last = word
    return result


def make_paragraphs_array(text=''):
    paragraphs_list = text.splitlines()
    paragraphs = []
    for p in paragraphs_list:
        parses = parser.parse(p)['parses']
        sentences = []
        for sentence in parses:
            sentences.append(sentence['text'])
        sentences = list(map(lambda s: split_words(s), sentences),)
        paragraphs.append(sentences)
    return(paragraphs)

In [2]:
import pandas as pd
common_words = pd.read_excel('common_words.ods')

In [9]:
common_words[common_words['word'].str.contains("'") == True]

,lemRank,lemma,PoS,lemFreq,wordFreq,word
4,2,be,v,32394756,6303682,'s
8,2,be,v,32394756,1957882,'re
10,2,be,v,32394756,1716283,'m
33,11,have,v,10514314,1015706,'ve
34,11,have,v,10514314,784592,'d
54,20,n't,x,5285354,5285354,n't
104,47,will,v,2372215,992384,'ll
1700,709,everybody,p,136787,53,ever'body
8244,3583,ma'am,n,19916,19899,ma'am
8245,3583,ma'am,n,19916,17,ma'ams


In [99]:
### construct map for words ocurrenses for a given text

def make_words_map(nestet_text = []):
    text_data = {}
    p_index = 0
    s_index = 0
    for paragraph in nestet_text:
        for sentence in paragraph:
            for word in  sentence:
                if re.match('\w+', word):
                    rout = {}
                    rout['paragraph'] = p_index
                    rout['sentence'] = s_index
                    lemma = common_words[common_words['word'] == word.lower()]['lemma'].unique()
                    if len(lemma) > 0:
                        lemma = lemma.item()
                        if lemma in text_data:
                            text_data[lemma].append(rout)
                        else:
                            text_data[lemma] = []
                            text_data[lemma].append(rout)
            s_index += 1
        p_index += 1
    return text_data

In [100]:
hiper_text = {}
nested = make_paragraphs_array(txt)
hiper_text['text'] = txt
hiper_text['words_map'] = make_words_map(nested)
hiper_text['nested'] = nested


isn


In [101]:
pprint(hiper_text)

{'nested': [[['This',
              ' ',
              'is',
              ' ',
              'an',
              ' ',
              'example',
              ' ',
              'sentence',
              ',',
              ' ',
              'ok',
              '?']],
            [],
            [['And', ',', ' ', 'see', '!'],
             ['This',
              ' ',
              'is',
              ' ',
              'another',
              ' ',
              '(',
              'example',
              ' ',
              'sentence',
              ')',
              ',',
              ' ',
              'is',
              "n't",
              ' ',
              'more',
              '.']]],
 'text': 'This is an example sentence, ok?\n'
         '\n'
         "And, see! This is another (example sentence), isn't more.",
 'words_map': {'OK': [{'paragraph': 0, 'sentence': 0}],
               'a': [{'paragraph': 0, 'sentence': 0}],
               'and': [{'paragraph': 2, 'sentence': 1}],
